# UDFs and Random Data

In [20]:
import iarray as ia
import numpy as np
import seaborn as sns

%load_ext memprofiler

sns.set_theme()

The memprofiler extension is already loaded. To reload it, use:
  %reload_ext memprofiler


In [21]:
shape = (10_000, 10_000)
size = np.prod(shape)

chunks = (1 * 1024, 1 * 1024)
blocks = (128, 128)

ia.set_config_defaults(favor=ia.Favor.SPEED, chunks=chunks, blocks=blocks, dtype=np.float32)

Config(codec=<Codec.LZ4: 1>, zfp_meta=0, clevel=9, favor=<Favor.SPEED: 1>, filters=[<Filter.SHUFFLE: 1>], fp_mantissa_bits=0, use_dict=False, nthreads=12, eval_method=<Eval.AUTO: 1>, seed=1, random_gen=<RandomGen.MRG32K3A: 0>, btune=True, dtype=<class 'numpy.float32'>, np_dtype=None, split_mode=<SplitMode.AUTO_SPLIT: 3>, chunks=(1024, 1024), blocks=(128, 128), urlpath=None, mode='w-', contiguous=None)

In [22]:
%%time

rand_data = ia.random.random_sample(shape, fp_mantissa_bits=15)

CPU times: user 715 ms, sys: 260 ms, total: 974 ms
Wall time: 146 ms


In [23]:
from iarray import udf

@udf.jit
def circle(out: udf.Array(udf.float32, 2), vals: udf.Array(udf.float32, 2), nrows: udf.int64, ncols: udf.int64) -> udf.int32:
    n = out.window_shape[0]
    m = out.window_shape[1]
    row_start = out.window_start[0]
    col_start = out.window_start[1]
    for i in range(n):
        for j in range(m):
            x = (2. * (row_start + i) / nrows) - 1.
            y = (2. * (col_start + j) / ncols) - 1.
            if (x ** 2 + y ** 2) < 1:
                out[i, j] = vals[i, j]
            else:
                out[i, j] = 0
    return 0

In [24]:
%%mprof_run iarray_circle

expr = ia.expr_from_udf(circle, [rand_data], list(shape), shape=shape, dtype=np.float32)
ia_circle = expr.eval()

memprofiler: used 150.57 MiB RAM (peak of 150.85 MiB) in 0.3226 s, total RAM usage 1709.52 MiB


In [25]:
#ia_out.data

In [26]:
#ax = sns.heatmap(ia_circle.data)

In [27]:
ia.udf_registry.clear()
@udf.scalar()
def filter(val: udf.float32, x: udf.float32, y: udf.float32) -> udf.float32:
    if (x ** 2 + y ** 2) < 1:
        return val
    return 0


In [28]:
@udf.jit
def circle_lib(out: udf.Array(udf.float32, 2), vals: udf.Array(udf.float32, 2), nrows: udf.int64, ncols: udf.int64) -> udf.int32:
    n = out.window_shape[0]
    m = out.window_shape[1]
    row_start = out.window_start[0]
    col_start = out.window_start[1]
    for i in range(n):
        for j in range(m):
            x = (2. * (row_start + i) / nrows) - 1.
            y = (2. * (col_start + j) / ncols) - 1.
            out[i, j] = ulib.filter(vals[i, j], x, y)
    return 0

In [29]:
%%mprof_run iarray_circle2

expr = ia.expr_from_udf(circle_lib, [rand_data], list(shape), shape=shape, dtype=np.float32)
ia_circle_lib = expr.eval()

memprofiler: used 80.12 MiB RAM (peak of 80.40 MiB) in 0.2982 s, total RAM usage 1790.90 MiB


In [30]:
%mprof_plot iarray_circle iarray_circle2

In [31]:
%%time
area_circle = ia.sum(ia_circle)

CPU times: user 260 ms, sys: 20.9 ms, total: 281 ms
Wall time: 50 ms


In [32]:
%%time
area_circle_lib = ia.sum(ia_circle_lib)
print(f"Diff in circle area: {(area_circle - area_circle_lib) / area_circle}")

Diff in circle area: 0.0
CPU times: user 285 ms, sys: 22.1 ms, total: 308 ms
Wall time: 49.1 ms


In [33]:
%%time
area_square = ia.sum(rand_data)

CPU times: user 328 ms, sys: 22.2 ms, total: 351 ms
Wall time: 55 ms


In [37]:
print(f"PI value: {4 * area_circle / area_square}")

PI value: 3.1415839572209627
